In [1]:
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill
from copy import copy

# --- CONFIGURATION ---
file_path = "MAXQDA24 Document comparison chart.xlsx"  # change to your actual file path
filtered_file = "filtered_file.xlsx"

# Define the desired hex codes (without the '#' character) in uppercase.
# (Your hex codes are given as: green: #32BD59, gray: #78909C)
GREEN_HEX = "32BD59"
GRAY_HEX = "78909C"

# --- FUNCTIONS ---
def get_cell_color(cell):
    """
    Returns the 6-digit hex color string (in uppercase) for a cell's fill,
    if it is a solid fill. If an alpha channel is present (8 characters), it is stripped.
    Returns None if no appropriate fill is found.
    """
    fill = cell.fill
    if fill is None:
        return None
    # Only process solid fills (other fill types may require different handling)
    if fill.fill_type != "solid":
        return None

    rgb = None
    # Try the foreground color
    if fill.fgColor is not None and fill.fgColor.rgb is not None:
        rgb = fill.fgColor.rgb
    # Fallback to start_color if necessary
    elif fill.start_color is not None and fill.start_color.rgb is not None:
        rgb = fill.start_color.rgb

    if rgb:
        # Remove any alpha channel (if the color is in the form "FF32BD59", keep the last 6 digits)
        if len(rgb) == 8:
            rgb = rgb[-6:]
        return rgb.upper()
    return None

# --- LOAD THE ORIGINAL WORKBOOK ---
wb = load_workbook(file_path)
ws = wb.active  # using the active (first) sheet

# --- FILTER ROWS BASED ON COLOR ---
# We assume that only the numbered cells (columns 2 onward) are colored.
filtered_rows = []  # this will store the row of cell objects (not just their values)
for row in ws.iter_rows(min_row=2):  # skip header row (assumed row 1)
    row_colors = set()
    # Check cells starting with the second column (index 1)
    for cell in row[1:]:
        color = get_cell_color(cell)
        if color is not None:
            row_colors.add(color)
    # Uncomment the next line to debug which colors are detected per row:
    # print(f"Row {row[0].row} detected colors: {row_colors}")
    if GREEN_HEX in row_colors and GRAY_HEX in row_colors:
        filtered_rows.append(row)

# --- CREATE A NEW WORKBOOK AND COPY THE DATA WITH FORMATTING ---
new_wb = Workbook()
new_ws = new_wb.active

# --- Copy the header row (assumed to be row 1) with formatting ---
for col_index, cell in enumerate(ws[1], start=1):
    new_cell = new_ws.cell(row=1, column=col_index, value=cell.value)
    if cell.has_style:
        new_cell.font = copy(cell.font)
        new_cell.border = copy(cell.border)
        new_cell.fill = copy(cell.fill)
        new_cell.number_format = cell.number_format
        new_cell.protection = copy(cell.protection)
        new_cell.alignment = copy(cell.alignment)

# --- Copy each filtered row with formatting ---
new_row_index = 2  # start copying from row 2 in the new workbook
for row in filtered_rows:
    for col_index, cell in enumerate(row, start=1):
        new_cell = new_ws.cell(row=new_row_index, column=col_index, value=cell.value)
        if cell.has_style:
            new_cell.font = copy(cell.font)
            new_cell.border = copy(cell.border)
            new_cell.fill = copy(cell.fill)
            new_cell.number_format = cell.number_format
            new_cell.protection = copy(cell.protection)
            new_cell.alignment = copy(cell.alignment)
    new_row_index += 1

# --- SAVE THE NEW WORKBOOK ---
new_wb.save(filtered_file)
print(f"Filtered data (with formatting) saved to {filtered_file}")


Filtered data (with formatting) saved to filtered_file.xlsx
